In [40]:
from collections import Counter

import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np

import json
from ast import literal_eval  # OP = OP, NP = P (by Articuno's Law)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression

%matplotlib inline

In [14]:
df = pd.read_json('data/recipes.json')

In [15]:
df.submitted = df.submitted.astype('datetime64')

In [16]:
# Recipes which take more than 5 hours
df = df[df['minutes'] < 300]

In [17]:
# We don't want oven or dessert recipes
index = df.tags.to_numpy()
f = lambda x: ('desserts' not in x and 'oven' not in x)
index = [f(x) for x in index]
df = df[index]

# Create vocabulary

In [18]:
df['ings'] = df.ingredients.apply(lambda x: ' '.join(['_'.join(ing.split(' ')) for ing in x]))

In [48]:
docs = df.ings.to_numpy()

In [49]:
vectorizer = CountVectorizer(analyzer='word', tokenizer=lambda x: x.split(' '))

In [50]:
X = vectorizer.fit_transform(docs).toarray()

In [51]:
X = X.astype(bool).astype(int)

In [52]:
Y = ((df.std_rating - df.std_rating.mean()) / df.std_rating.std()).to_numpy()

In [53]:
Y

array([-0.58412637, -0.58412637, -0.58412637, ..., -0.58412637,
       -0.58412637, -0.58412637])

In [54]:
feat_names = vectorizer.get_feature_names()

# Apply linear regression to dataset

In [55]:
model = LinearRegression()

In [ ]:
model.fit(X, Y)

In [ ]:
ing_contr = dict(zip(feat_names, model.coef_))

In [ ]:
ing_contr = dict(sorted(ing_contr.items(), key=lambda kv: kv[1], reverse=True))

In [ ]:
ing_contr